### Описание проекта.

Задачей данного проекта является создание прототипа приложения, которое сможет выполнять функцию оптимизации. Нам предоставлены данные с разными пищевыми продуктами с указанными ценами, а также содержанием в продуктах микроэлементов. У человека есть суточная потребность в различных микроэлементах, удовлетворить которую можно с помощью сочитания различных продуктов. Нашей целью является подобрать такой рацион, при котором будет удовлетворена суточная потребность в микроэлементах, а затраты на покупку продуктов будут минимальными.

### Принцип работы приложения на кратком примере.

Решать задачу будем с помощью бесплатной библиотеки PuLP, предназначенной для решения задач оптимизации с помощью методов линейного программирования.

In [2]:
from pulp import *
import pandas as pd

In [2]:
# Суточная потребность в граммах

test_micronutrients = pd.Series({'Ca': 0.8,
                            'Mg': 0.35,
                            'P': 0.8}, 
                            name='Суточная норма')
test_micronutrients

Ca    0.80
Mg    0.35
P     0.80
Name: Суточная норма, dtype: float64

In [3]:
# Содержание микроэлементов в 100г продукта

test_data = pd.DataFrame({'Гречка': [0.02, 0.2, 0.298, 89],
                          'Сыр голландский': [1, 0.05, 0.54, 500]}, index=['Ca', 'Mg', 'P', 'Цена'])

In [4]:
test_data.join(test_micronutrients).transpose()

,Ca,Mg,P,Цена
Гречка,0.02,0.20,0.298,89.0
Сыр голландский,1.00,0.05,0.540,500.0
Суточная норма,0.80,0.35,0.800,NaN


Пусть гречка = x1 (вернее сказать - неизвестная доля гречки), а сыр = x2, тогда ограничения примут следующий вид:
* Содержание кальция: $0.02 * x1 + 1 * x2 >= 0.8$
* Содержание магния: $0.2 * x1 + 0.05 * x2 >= 0.35$
* Содержание фосфора: $0.298 * x1 + 0.54 * x2 >= 0.8$

А целевая функция (цена) будет:
$f(x) = 89 * x1 + 500 * x2 → min$

In [5]:
# Создаем проблему оптимизации

prob = LpProblem("Minimize_cost", LpMinimize)

In [6]:
# Определяем переменные, каждая переменная соответствует одному товару

x1 = pulp.LpVariable('гречка', lowBound=0)
x2 = pulp.LpVariable('сыр', lowBound=0)

In [7]:
# Переменная prob начинает сбор данных с помощью оператора +=

prob += 89 * x1 + 500 * x2, 'Целевая функция'

prob += 0.02 * x1 + 1 * x2 >= 0.8, "Содержание кальция должно быть больше 0.8"
prob += 0.2 * x1 + 0.05 * x2 >= 0.35, "Содержание магния должно быть больше 0.35"
prob += 0.298 * x1 + 0.54 * x2 >= 0.8, "Содержание фосфора должно быть больше 0.8"

In [8]:
prob.solve()

1

In [9]:
print(value(prob.objective))

523.0653221


In [10]:
for variable in prob.variables():
    print(variable.name, "=", variable.varValue)

гречка = 1.5577889
сыр = 0.76884422


Таким образом, для удовлетворения суточной потребности в K, Mg, P и минимальных трат на выбранные продукты, нам необходимо купить 156г. гречки и 77г. сыра и потратить при этом 523 руб.

---

### План проекта.
 * Составить датафрейм с продуктами, пищевой ценностью и ценами. 
   * Для начала это будет небольшая таблица, составленная вручную (100 наиболее популярных продуктовых товаров) 
   * В дальнейшем необходимо настроить автоматический парсинг и мониторинг цен разных розничных сетей.
 * Рассчет индивидуальной потребности человека в микроэлементах по указанным параметрам.
 * Написание кода программы и отладка.
 * Создание телеграм-бота.
 * Презентация проекта.
 * Запуск стартапа 😁.

----

### Загрузка датасета.

Датасет собран на основе данных [Министерства сельского хозяйства США](https://fdc.nal.usda.gov/)

In [3]:
data = pd.read_csv('nutrient_data(copy2).csv')

In [4]:
data.drop(['Unnamed: 0', 'Description'], axis='columns', inplace=True)

In [5]:
obj_col = data.columns.drop(['DescriptionRu', 'FoodCategory', 'PriceRUB100g '])
data[obj_col] = data[obj_col].apply(lambda x: x.str.replace(',', '.').astype('float64'))

In [6]:
pd.set_option('display.max_columns', None)
data.head()

,DescriptionRu,PriceRUB100g,FoodCategory,Protein,Carbohydrate,Fat,Total Fiber,Magnesium (Mg),SFA,Iron (Fe),Selenium (Se),Vitamin K (phylloquinone),Vitamin B6 (pyridoxine),Copper (Cu),Phosphorus (P),Linoleic 18:2 Omega-6,Calcium,Potassium (K),Manganese (Mn),Sodium (Na),Folate,Thiamin,Zinc,Vitamin E (alpha-tocopherol),Niacin,Biotin,Riboflavin (Vitamin B2),Vitamin B12,Vitamin A,Vitamin D,EPA 20:5 Omega-3,ALA 18:3 Omega-3,Arachidonic 20:4 Omega-6,DHA 22:6 Omega-3,Trans Fat,Iodine (I),Molybdenum (Mo),Vitamin C,Choline,Pantothenic Acid (Vitamin B5),Kcal
0,Хлеб белый,10,Baked Products,9.43000,49.20000,3.590,2.3,26.9,0.821,3.360,23.2,0.0,0.0920,124.0,113.0,1.670,211.00,117.0,0.632,477.000,0.00,0.507,0.880,0.00,4.760,0.00,0.2400,0.00,0.0,0.0,0.002,0.200,0.002,0.000,0.035,0.0,0.0,0.0,0.0,0.548,266.830
1,Хлеб цельнозерновой,17,Baked Products,12.30000,43.10000,3.550,6.0,76.6,0.732,2.560,25.8,0.0,0.2160,226.0,212.0,1.460,163.00,250.0,2.180,450.000,42.00,0.391,1.760,2.82,4.430,0.00,0.1660,0.00,0.0,0.0,0.000,0.139,0.004,0.000,0.022,0.0,0.0,0.0,27.2,0.650,253.550
2,Печенье овсяное с изюмом,80,Baked Products,5.79000,69.60000,14.300,3.3,32.4,4.780,2.280,6.0,17.1,0.0700,141.0,111.0,4.210,29.00,245.0,0.755,314.000,34.00,0.295,0.620,2.69,2.110,0.00,0.2180,0.00,0.0,0.0,0.004,0.632,0.007,0.001,0.040,0.0,0.0,0.0,0.0,0.400,430.260
3,Говядина корейка,50,Beef Products,22.80000,0.00000,6.390,0.0,11.3,2.560,1.900,21.3,0.0,0.5750,43.0,206.0,0.259,15.00,282.0,0.002,45.000,0.00,0.053,3.760,0.25,6.880,0.00,0.2020,1.72,0.0,0.0,0.003,0.009,0.048,0.000,0.306,0.0,0.0,0.0,56.3,0.360,148.710
4,Мука овсяная цельнозерновая,20,Cereal Grains and Pasta,13.16875,69.91725,6.309,10.5,125.3,0.000,3.997,38.2,0.0,0.1476,443.4,371.8,0.000,42.81,373.1,3.257,3.619,34.69,0.390,2.754,0.00,1.945,20.15,0.1613,0.00,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.0,125.4,0.0,0.0,0.000,389.125


----

### Тест 1 - Рассчет стоимости для выживания с учетом необходимого количества белков и килокалорий.

Рассчеты на моем примере: Возраст - 25, Рост - 187, Вес - 85, умеренная активность.

In [15]:
# Необходимое количество микроэлементов в суточном рационе

micronutrients = pd.Series({'Kcal': 3376,
                            'Protein': 68,
                            'Vitamin A': 900,
                            'Vitamin C': 90,
                            'Vitamin B6': 1.3,
                            'Vitamin E': 15,
                            'Vitamin K': 120,
                            'Thiamin': 1.2,
                            'Vitamin B12': 2.4,
                            'Riboflavin': 1.3,
                            'Folate': 400,
                            'Niacin': 16,
                            'Choline': 550,
                            'Pantothenic Acid': 5,
                            'Biotin': 30}, 
                            name='Суточная норма')
micronutrients

Kcal                3376.0
Protein               68.0
Vitamin A            900.0
Vitamin C             90.0
Vitamin B6             1.3
Vitamin E             15.0
Vitamin K            120.0
Thiamin                1.2
Vitamin B12            2.4
Riboflavin             1.3
Folate               400.0
Niacin                16.0
Choline              550.0
Pantothenic Acid       5.0
Biotin                30.0
Name: Суточная норма, dtype: float64

In [7]:
# Создаем объекты-переменные, представляющие собой долю опредленного продукта: x1, x2...x88

dict_var = dict(zip(['x' + str(i) for i in range(89)], list(data['DescriptionRu'])))

for i in dict_var:
    dict_var[i] = pulp.LpVariable(dict_var[i], lowBound=0)

In [8]:
# Задача - Минимизация стоимости продуктов

prob = LpProblem("Minimize_cost", LpMinimize)

In [17]:
# Функция для создания математической модели задачи

# Целевая функция

def target_function(feature):
    func_list = []
    i = 0
    for key in dict_var.keys():
        step = dict_var[key] * data[feature][i]
        func_list.append(step)
        i += 1
    return sum(func_list), 'Целевая функция'


# Ограничения

def limitation(feature, value, name, is_greater=True):
    func_list = []
    i = 0
    for key in dict_var.keys():
        step = dict_var[key] * data[feature][i]
        func_list.append(step)
        i += 1
    if is_greater is True:
        return sum(func_list) >= value, name
    else:
        return sum(func_list) <= value, name

In [20]:
prob += target_function('PriceRUB100g ')
prob += limitation('Kcal', 3376, 'Каллорийность рациона должна превышать 3376 ккал')
prob += limitation('Protein', 68, 'Количество белка должно быть больше 68г')

In [21]:
prob.solve()

print(f'Стоимость продуктов составит: {value(prob.objective):.4} руб')

Стоимость продуктов составит: 44.15 руб


In [22]:
for variable in prob.variables():
    if variable.varValue != 0:
        print(variable.name, "=", round(variable.varValue * 100, 1), 'г')

Масло_подсолнечное = 158.1 г
Мука_пшеничная = 566.7 г


*__Самый дешевый способ выжить - Кушать лепешки!__*

----